#### How to search in OpenSearch 

### 1. Import Libraries:

In [3]:
import pandas as pd
from opensearchpy import OpenSearch
from opensearchpy.client.indices import IndicesClient
import json

### 2. Create Client OpenSearch 

In [41]:
opensearch_client =OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

### 3. Define Mapping for an index.

In [42]:
index_mapping = {
  "mappings": {
    "properties": {
      "genres": {
        "type": "keyword"
      },
        "title": {
        "type": "text"
        }
    }
  }
}

For our demo purpose we will use the NintendoGames dataset. The dataset contains 9 columns. However, we will only use the title and genres columns for our demo. We will define the genres column as a keyword and the title column as a text.

### 4. Create an index

In [43]:
opensearch_client.indices.create(index='nintendo_games', body=index_mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nintendo_games'}

### 5. Check if the index was created

In [44]:
opensearch_client.indices.exists(index='nintendo_games')

True

In [40]:
opensearch_client.indices.delete(index='nintendo_games')

{'acknowledged': True}

### 6. Load the dataset and transform it to json

In [45]:
dataset = pd.read_csv('NintendoGames.csv')

In [46]:
dataset['genres'] = dataset['genres'].transform(lambda x: x.strip('[]').replace("'", "").split(', '))

In [47]:
dataset = json.loads(dataset.to_json(orient='records'))

In [48]:
dataset[0]

{'meta_score': None,
 'title': 'Super Mario RPG',
 'platform': 'Switch',
 'date': 'Nov 17, 2023',
 'user_score': None,
 'link': '/game/switch/super-mario-rpg',
 'esrb_rating': 'E',
 'developers': "['Nintendo']",
 'genres': ['Role-Playing', 'Japanese-Style']}

### 7. Create a bulk operation

During Bulk operation, it is possible to do multiple operations in a single request such as:
* create - to create a document if it doesn’t already exist and returns an error otherwise
* delete - to delete a document
* index - to create a document if it doesn’t already exist and returns an error otherwise
* update - to update existing documents and returns an error if the document doesn’t exist

In our case, we will create a bulk operation to index all the documents in the dataset by using index operation.

In [49]:
def create_bulk_opensearch(index_name, dataset):
    bulk_opensearch = ''
    for row in dataset:
        bulk_opensearch += '{"index": {"_index": "%s"}}\n' % index_name
        bulk_opensearch += json.dumps(row) + '\n'
    return bulk_opensearch

bulk_opensearch = create_bulk_opensearch('nintendo_games', dataset)

after transforming dataset for preparing bulk operation, we will get the following result:

In [52]:
bulk_opensearch

'{"index": {"_index": "nintendo_games"}}\n{"meta_score": null, "title": "Super Mario RPG", "platform": "Switch", "date": "Nov 17, 2023", "user_score": null, "link": "/game/switch/super-mario-rpg", "esrb_rating": "E", "developers": "[\'Nintendo\']", "genres": ["Role-Playing", "Japanese-Style"]}\n{"index": {"_index": "nintendo_games"}}\n{"meta_score": null, "title": "WarioWare: Move It!", "platform": "Switch", "date": "Nov 3, 2023", "user_score": null, "link": "/game/switch/warioware-move-it!", "esrb_rating": "RP", "developers": "[\'Intelligent Systems\']", "genres": ["Miscellaneous", "Party / Minigame"]}\n{"index": {"_index": "nintendo_games"}}\n{"meta_score": null, "title": "Super Mario Bros. Wonder", "platform": "Switch", "date": "Oct 20, 2023", "user_score": null, "link": "/game/switch/super-mario-bros-wonder", "esrb_rating": "E", "developers": "[\'Nintendo\']", "genres": ["Action", "Platformer", "2D"]}\n{"index": {"_index": "nintendo_games"}}\n{"meta_score": null, "title": "Detectiv

### 8. Index the dataset

In [53]:
response_bulk = opensearch_client.bulk(bulk_opensearch)

### 9. Check if the dataset was indexed

In [54]:
response_bulk

{'took': 128,
 'errors': False,
 'items': [{'index': {'_index': 'nintendo_games',
    '_id': '98PlZIoBqjgNgJpQ8cLP',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1094,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'nintendo_games',
    '_id': '-MPlZIoBqjgNgJpQ8cLP',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1095,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'nintendo_games',
    '_id': '-cPlZIoBqjgNgJpQ8cLP',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1096,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'nintendo_games',
    '_id': '-sPlZIoBqjgNgJpQ8cLP',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1097,
    '_primary_term': 1,


Check if there are any errors

In [51]:
response_bulk['errors']

False

In [13]:
index_mapping =  IndicesClient(opensearch_client).get_mapping(index='nintendo_games')

In [146]:
index_mapping

{'nintendo_games': {'mappings': {'properties': {'date': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'developers': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'esrb_rating': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'genres': {'type': 'keyword'},
    'link': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'meta_score': {'type': 'float'},
    'platform': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'title': {'type': 'text'},
    'user_score': {'type': 'float'}}}}}

In [147]:
title_mapping = index_mapping['nintendo_games']['mappings']['properties']['title']

In [148]:
title_mapping

{'type': 'text'}

Full Text Search

In [55]:
search_1 = opensearch_client.search(body={
    "query": {
        "match": {
            "title": "Mario"
        }
    }
})

In [56]:
search_1

{'took': 107,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 302, 'relation': 'eq'},
  'max_score': 2.5115013,
  'hits': [{'_index': 'nintendo_games',
    '_id': '5Dq4XIoBTw-2vdEqo5HK',
    '_score': 2.5115013,
    '_source': {'meta_score': None,
     'title': 'Mario Tennis',
     'platform': 'WII',
     'date': 'Jun 28, 2010',
     'user_score': None,
     'link': '/game/wii/mario-tennis',
     'esrb_rating': 'E',
     'developers': "['Camelot Software Planning']",
     'genres': "['Sports', 'Traditional', 'Tennis']"}},
   {'_index': 'nintendo_games',
    '_id': 'STq4XIoBTw-2vdEqo5LK',
    '_score': 2.5115013,
    '_source': {'meta_score': None,
     'title': 'Mario Calculator',
     'platform': 'DS',
     'date': 'Jun 15, 2009',
     'user_score': 6.3,
     'link': '/game/ds/mario-calculator',
     'esrb_rating': None,
     'developers': "['Nintendo']",
     'genres': "['Action', 'Miscellaneous', 'General', 'Edu

In [82]:
search_2 = opensearch_client.search(body={
    "query": {
        "match": {
            "title": {
                "query": "Maroi",
                "fuzziness": "AUTO"
            }
        }
    }
})

In [83]:
search_2

{'took': 18,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 302, 'relation': 'eq'},
  'max_score': 2.009201,
  'hits': [{'_index': 'nintendo_games',
    '_id': '5Dq4XIoBTw-2vdEqo5HK',
    '_score': 2.009201,
    '_source': {'meta_score': None,
     'title': 'Mario Tennis',
     'platform': 'WII',
     'date': 'Jun 28, 2010',
     'user_score': None,
     'link': '/game/wii/mario-tennis',
     'esrb_rating': 'E',
     'developers': "['Camelot Software Planning']",
     'genres': "['Sports', 'Traditional', 'Tennis']"}},
   {'_index': 'nintendo_games',
    '_id': 'STq4XIoBTw-2vdEqo5LK',
    '_score': 2.009201,
    '_source': {'meta_score': None,
     'title': 'Mario Calculator',
     'platform': 'DS',
     'date': 'Jun 15, 2009',
     'user_score': 6.3,
     'link': '/game/ds/mario-calculator',
     'esrb_rating': None,
     'developers': "['Nintendo']",
     'genres': "['Action', 'Miscellaneous', 'General', 'Edutain

In [86]:
search_3 = opensearch_client.search(body={
    "query": {
        "multi_match": {
            "query": "*gent",
            "fields": ["title^2", "developers^1"],
            "fuzziness": "3"
        }
    }
})

In [87]:
search_3

{'took': 24,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 60, 'relation': 'eq'},
  'max_score': 12.522595,
  'hits': [{'_index': 'nintendo_games',
    '_id': '_zq4XIoBTw-2vdEqo5LL',
    '_score': 12.522595,
    '_source': {'meta_score': 87.0,
     'title': 'Elite Beat Agents',
     'platform': 'DS',
     'date': 'Nov 6, 2006',
     'user_score': 8.5,
     'link': '/game/ds/elite-beat-agents',
     'esrb_rating': 'E10+',
     'developers': "['iNiS']",
     'genres': "['Action', 'Miscellaneous', 'Rhythm', 'Music', 'Music']"}},
   {'_index': 'nintendo_games',
    '_id': 'STrCXIoBTw-2vdEq9Jeh',
    '_score': 12.522595,
    '_source': {'meta_score': 87.0,
     'title': 'Elite Beat Agents',
     'platform': 'DS',
     'date': 'Nov 6, 2006',
     'user_score': 8.5,
     'link': '/game/ds/elite-beat-agents',
     'esrb_rating': 'E10+',
     'developers': "['iNiS']",
     'genres': "['Action', 'Miscellaneous', 'Rhythm', 

Term Search

In [149]:
genres_mapping = index_mapping['nintendo_games']['mappings']['properties']['genres']

In [150]:
genres_mapping

{'type': 'keyword'}

In [22]:
search_4 = opensearch_client.search(body={
    "query": {
        "term": {
            "genres":{
                "value": "Miscellaneous",
                 "case_insensitive": True
            }
        }
    }
})

In [23]:
search_4

{'took': 29,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 291, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'nintendo_games',
    '_id': 'HjoyXYoBTw-2vdEqh5jC',
    '_score': 1.0,
    '_source': {'meta_score': None,
     'title': 'WarioWare: Move It!',
     'platform': 'Switch',
     'date': 'Nov 3, 2023',
     'user_score': None,
     'link': '/game/switch/warioware-move-it!',
     'esrb_rating': 'RP',
     'developers': "['Intelligent Systems']",
     'genres': ['Miscellaneous', 'Party / Minigame']}},
   {'_index': 'nintendo_games',
    '_id': 'IzoyXYoBTw-2vdEqh5jD',
    '_score': 1.0,
    '_source': {'meta_score': None,
     'title': 'Pokemon Sleep',
     'platform': 'iOS',
     'date': 'Jul 20, 2023',
     'user_score': None,
     'link': '/game/ios/pokemon-sleep',
     'esrb_rating': None,
     'developers': "['The Pokemon Company', ' Select Button']",
     'genres': ['Role-Playing', 'Miscel

In [161]:
search_5 = opensearch_client.search(body={
    "query": {
        "wildcard": {
            "genres":{
                "value": "Miscell*",
                 "case_insensitive": False
            }
        }
    }
})

In [162]:
search_5

{'took': 13,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 291, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'nintendo_games',
    '_id': 'HjoyXYoBTw-2vdEqh5jC',
    '_score': 1.0,
    '_source': {'meta_score': None,
     'title': 'WarioWare: Move It!',
     'platform': 'Switch',
     'date': 'Nov 3, 2023',
     'user_score': None,
     'link': '/game/switch/warioware-move-it!',
     'esrb_rating': 'RP',
     'developers': "['Intelligent Systems']",
     'genres': ['Miscellaneous', 'Party / Minigame']}},
   {'_index': 'nintendo_games',
    '_id': 'IzoyXYoBTw-2vdEqh5jD',
    '_score': 1.0,
    '_source': {'meta_score': None,
     'title': 'Pokemon Sleep',
     'platform': 'iOS',
     'date': 'Jul 20, 2023',
     'user_score': None,
     'link': '/game/ios/pokemon-sleep',
     'esrb_rating': None,
     'developers': "['The Pokemon Company', ' Select Button']",
     'genres': ['Role-Playing', 'Miscel

Compound queries

In [165]:
search_6 = opensearch_client.search(body={
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "title": "Mario"
                    }
                },
                ],
            "must_not": [
                {
                    "match": {
                        "genres": "Miscellaneous"
                    }
                }
            ]
        }
    }
})

In [166]:
search_6

{'took': 44,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 120, 'relation': 'eq'},
  'max_score': 2.509985,
  'hits': [{'_index': 'nintendo_games',
    '_id': 'dzoyXYoBTw-2vdEqh5rE',
    '_score': 2.509985,
    '_source': {'meta_score': None,
     'title': 'Mario Tennis',
     'platform': 'WII',
     'date': 'Jun 28, 2010',
     'user_score': None,
     'link': '/game/wii/mario-tennis',
     'esrb_rating': 'E',
     'developers': "['Camelot Software Planning']",
     'genres': ['Sports', 'Traditional', 'Tennis']}},
   {'_index': 'nintendo_games',
    '_id': 'CDoyXYoBTw-2vdEqh5vE',
    '_score': 2.509985,
    '_source': {'meta_score': None,
     'title': 'Mario Golf',
     'platform': 'WII',
     'date': 'Oct 6, 2008',
     'user_score': None,
     'link': '/game/wii/mario-golf',
     'esrb_rating': 'E',
     'developers': "['Camelot Software Planning']",
     'genres': ['Sports', 'Traditional', 'Golf', 'Arcade']}

In [16]:
search_7 = opensearch_client.search(body={
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "title": "Mario"
                    }
                },
                {
                    "match": {
                        "genres": "Miscellaneous"
                    }
                }
            ]
        }
    }
})

In [17]:
search_7

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 31, 'relation': 'eq'},
  'max_score': 4.335801,
  'hits': [{'_index': 'nintendo_games',
    '_id': '3DoyXYoBTw-2vdEqh5rE',
    '_score': 4.335801,
    '_source': {'meta_score': None,
     'title': 'Mario Calculator',
     'platform': 'DS',
     'date': 'Jun 15, 2009',
     'user_score': 6.3,
     'link': '/game/ds/mario-calculator',
     'esrb_rating': None,
     'developers': "['Nintendo']",
     'genres': ['Action', 'Miscellaneous', 'General', 'Edutainment']}},
   {'_index': 'nintendo_games',
    '_id': '3ToyXYoBTw-2vdEqh5rE',
    '_score': 4.335801,
    '_source': {'meta_score': None,
     'title': 'Mario Clock',
     'platform': 'DS',
     'date': 'Jun 15, 2009',
     'user_score': 6.0,
     'link': '/game/ds/mario-clock',
     'esrb_rating': 'E',
     'developers': "['Nintendo']",
     'genres': ['Miscellaneous', 'General', 'General', 'Application']}

In [18]:
search_8 = opensearch_client.search(body={
  "query": {
    "match": {
      "text_entry": {
        "query": "Mario Party", 
        "minimum_should_match": "2"
      }
    }
  }
})

In [19]:
search_8

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [30]:
search_9 = opensearch_client.search(body={
  "query": {
    "match_phrase_prefix": {
      "text_entry": {
        "query": "Mari",
        "slop": 3
      }
    }
  }
})

In [31]:
search_9

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [33]:
search_10 = opensearch_client.search(body={
    "query": {
        "match": {
            "title": "Mario"
        }
    }
})

In [34]:
search_10

{'took': 15,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 151, 'relation': 'eq'},
  'max_score': 2.509985,
  'hits': [{'_index': 'nintendo_games',
    '_id': 'dzoyXYoBTw-2vdEqh5rE',
    '_score': 2.509985,
    '_source': {'meta_score': None,
     'title': 'Mario Tennis',
     'platform': 'WII',
     'date': 'Jun 28, 2010',
     'user_score': None,
     'link': '/game/wii/mario-tennis',
     'esrb_rating': 'E',
     'developers': "['Camelot Software Planning']",
     'genres': ['Sports', 'Traditional', 'Tennis']}},
   {'_index': 'nintendo_games',
    '_id': '3DoyXYoBTw-2vdEqh5rE',
    '_score': 2.509985,
    '_source': {'meta_score': None,
     'title': 'Mario Calculator',
     'platform': 'DS',
     'date': 'Jun 15, 2009',
     'user_score': 6.3,
     'link': '/game/ds/mario-calculator',
     'esrb_rating': None,
     'developers': "['Nintendo']",
     'genres': ['Action', 'Miscellaneous', 'General', 'Edutainmen